In [64]:
import requests
from bs4 import BeautifulSoup
import smtplib
import email.utils
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [68]:
def scrape(no_of_pages):
    base_url = 'https://news.ycombinator.com/news?p={}'
    links = list()
    subtexts = list()
    for i in range(1, no_of_pages+1):
        response = requests.get(base_url.format(i))
        soup = BeautifulSoup(response.text, 'html.parser')
        soup.prettify()
        links.extend(soup.select('.titlelink'))
        subtexts.extend(soup.select('.subtext'))
    return links, subtexts

def get_hacker_news(no_of_pages):
    links, subtexts = scrape(no_of_pages=no_of_pages)   
    hacker_news = list()
    for idx, item in enumerate(links):
        title = links[idx].getText()
        url = links[idx].get('href')
        vote = subtexts[idx].select('.score')
        if len(vote):
            score = int(vote[0].getText().replace(' points', ''))
            if score > 100:
                hacker_news.append({'title': title, 'url': url, 'score': score})
    hacker_news.sort(key=lambda x: x['score'], reverse=True)
    return hacker_news

def generate_html(hn):
    html = """\
    <html>
      <head></head>
      <body>
      <div>
        <ol>
    """
    for item in hn:
        html+= f'<li><a href="{item["url"]}">{item["title"]}({item["score"]} Votes)'
    html = html + """
        </ol>
      </div>
      </body>
    </html>
    """
    return html

def send_email(no_of_pages):
    # me == my email address
    # you == recipient's email address
    me = "pi@PiHoleJupyterMySQLACT"
    you = "sandeep.narayan.kv@gmail.com"
    
    # Get the news from scraping
    hn = get_hacker_news(no_of_pages)
    
    # text
    text = "Hi Sandeep!\nHere is the list of Hacker News Articles for the day to read\n"
    
    # generate html
    html = generate_html(hn)
    
    # Create message container - the correct MIME type is multipart/alternative.
    msg = MIMEMultipart('alternative')
    msg['Subject'] = "Hacker News - Today's Articles"
    msg["From"] = email.utils.formataddr(("Pi", me))
    msg["To"] = email.utils.formataddr(("Recipient", you))
    
    # Record the MIME types of both parts - text/plain and text/html.
    text_part = MIMEText(text, 'plain')
    html_part = MIMEText(html, 'html')

    # Attach parts into message container.
    # According to RFC 2046, the last part of a multipart message, in this case
    # the HTML message, is best and preferred.
    msg.attach(text_part)
    msg.attach(html_part)
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.set_debuglevel(True)
    server.login('sandi.narayan@gmail.com', 'izrmavxewrplsdoj')    
    server.sendmail(me, you, msg.as_string())
    server.quit()
    
if __name__ == '__main__':
    hn = send_email(5)

send: 'ehlo [127.0.1.1]\r\n'
reply: b'250-smtp.gmail.com at your service, [49.207.229.158]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [49.207.229.158]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'AUTH PLAIN AHNhbmRpLm5hcmF5YW5AZ21haWwuY29tAGl6cm1hdnhld3JwbHNkb2o=\r\n'
reply: b'235 2.7.0 Accepted\r\n'
reply: retcode (235); Msg: b'2.7.0 Accepted'
send: 'mail FROM:<pi@PiHoleJupyterMySQLACT> size=10566\r\n'
reply: b'250 2.1.0 OK k62sm7407891pga.86 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.0 OK k62sm7407891pga.86 - gsmtp'
send: 'rcpt TO:<sandeep.narayan.kv@gmail.com>\r\n'
reply: b'250 2.1.5 OK k62sm7407891p